In [79]:
stop_ids = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
# demand = {'A': 130, 'B': 48, 'C': 118, 'D': 115, 'E': 90, 'F': 62, 'G': 68}
stops = {'A': {'id': 'A', 'weight': 130, 'dist_to_wh': 632,
               'dist_to_stops': {'B': 661, 'C': 756, 'D': 1361, 'E': 1433, 'F': 1008, 'G': 374}},
         'B': {'id': 'B', 'weight': 48, 'dist_to_wh': 1081,
               'dist_to_stops': {'A': 661, 'C': 718, 'D': 1663, 'E': 1952, 'F': 1638, 'G': 1027}},
         'C': {'id': 'C', 'weight': 118, 'dist_to_wh': 628,
               'dist_to_stops': {'A': 756, 'B': 718, 'D': 982, 'E': 1422, 'F': 1331, 'G': 926}},
         'D': {'id': 'D', 'weight': 115, 'dist_to_wh': 741,
               'dist_to_stops': {'A': 1361, 'B': 1663, 'C': 982, 'E': 703, 'F': 1088, 'G': 1219}},
         'E': {'id': 'E', 'weight': 90, 'dist_to_wh': 872,
               'dist_to_stops': {'A': 1433, 'B': 1952, 'C': 1422, 'D': 703, 'F': 642, 'G': 1129}},
         'F': {'id': 'F', 'weight': 62, 'dist_to_wh': 707,
               'dist_to_stops': {'A': 1008, 'B': 1638, 'C': 1331, 'D': 1088, 'E': 642, 'G': 639}},
         'G': {'id': 'G', 'weight': 68, 'dist_to_wh': 499,
               'dist_to_stops': {'A': 374, 'B': 1027, 'C': 926, 'D': 1219, 'E': 1129, 'F': 639}}}
van_cap = 300

# distances = {'A': {'B': 661, 'C': 756, 'D': 1361, 'E': 1433, 'F': 1008, 'G': 374, 'DC': 632},
#              'B': {'A': 661, 'C': 718, 'D': 1663, 'E': 1952, 'F': 1638, 'G': 1027, 'DC': 1081},
#              'C': {'A': 756, 'B': 718, 'D': 982, 'E': 1422, 'F': 1331, 'G': 926, 'DC': 628},
#              'D': {'A': 1361, 'B': 1663, 'C': 982, 'E': 703, 'F': 1088, 'G': 1219, 'DC': 741},
#              'E': {'A': 1433, 'B': 1952, 'C': 1422, 'D': 703, 'F': 642, 'G': 1129, 'DC': 872},
#              'F': {'A': 1008, 'B': 1638, 'C': 1331, 'D': 1088, 'E': 642, 'G': 639, 'DC': 707},
#              'G': {'A': 374, 'B': 1027, 'C': 926, 'D': 1219, 'E': 1129, 'F': 639, 'DC': 499},
#              'DC': {'A': 632, 'B': 1081, 'C': 628, 'D': 741, 'E': 872, 'F': 707, 'G': 499}}

distances = {'A': {'B': 661, 'C': 756, 'D': 1361, 'E': 1433, 'F': 1008, 'G': 374},
             'B': {'C': 718, 'D': 1663, 'E': 1952, 'F': 1638, 'G': 1027},
             'C': {'D': 982, 'E': 1422, 'F': 1331, 'G': 926},
             'D': {'E': 703, 'F': 1088, 'G': 1219},
             'E': {'F': 642, 'G': 1129},
             'F': {'G': 639}}

distances_dc = {'A': 632, 'B': 1081, 'C': 628, 'D': 741, 'E': 872, 'F': 707, 'G': 499}

In [87]:
from collections import deque

class Route(object):

    max_load = van_cap

    def __init__(self, stop, measure):

        assert stop[measure] <= Route.max_load, "exceeds max_load"
        self.llist = deque([stop])
        self.measure = measure  # string used as key to dict
        self.load = stop[measure]
        self.spare_cap = Route.max_load - self.load

        self.first_stop = stop['id']
        self.last_stop = stop['id']
        self.first_and_last_stops = (self.first_stop, self.last_stop)

        self.n_stops = 1
        self.total_dist = stop['dist_to_wh'] * 2

    def merge(self, route):
        self.update()
        route.eliminate()

    def update(self):
        pass

    def eliminate(self):
        pass

    def new_first_stop(self, stop):

        assert stop[self.measure] <= self.spare_cap, "exceeds max_load"

        self.total_dist += stop['dist_to_wh'] - self.llist[0]['dist_to_wh'] \
                           + stop['dist_to_stops'][self.first_stop]

        self.llist.appendleft(stop)
        self.first_stop = stop['id']
        self.first_and_last_stops = (self.first_stop, self.last_stop)
        self.n_stops += 1
        self.load += stop[self.measure]
        self.spare_cap = Route.max_load - self.load


    def new_last_stop(self, stop):

        assert stop[self.measure] <= self.spare_cap, "exceeds max_load"

        self.total_dist += stop['dist_to_wh'] - self.llist[-1]['dist_to_wh'] \
                           + stop['dist_to_stops'][self.last_stop]

        self.llist.append(stop)
        self.n_stops += 1
        self.last_stop = stop['id']
        self.first_and_last_stops = (self.first_stop, self.last_stop)
        self.load += stop[self.measure]
        self.spare_cap = Route.max_load - self.load





# build the list of initial routes:
routes_list = [Route(stops[each], measure='weight') for each in stops]

In [89]:
routes_dict = {}

for each in stops:
    routes_dict[each] = []

for route in routes_list:
    for each in route.first_and_last_stops:
        if route not in routes_dict[each]:
            routes_dict[each].append(route)

# this needs to be updated incrementally after each merge

In [91]:
print(routes_dict['A'][0].llist)
print(routes_dict['A'][0].first_stop)
print(routes_dict['A'][0].last_stop)
print(routes_dict['A'][0].load)
print(routes_dict['A'][0].n_stops)
print(routes_dict['A'][0].total_dist)
#
# routes_dict['A'].new_first_stop(stops['B'])
# print(routes_dict['A'].first_stop)
# print(routes_dict['A'].last_stop)
# print(routes_dict['A'].load)
# print(routes_dict['A'].spare_cap)
# print(routes_dict['A'].n_stops)
# print(routes_dict['A'].total_dist)

deque([{'id': 'A', 'weight': 130, 'dist_to_wh': 632, 'dist_to_stops': {'B': 661, 'C': 756, 'D': 1361, 'E': 1433, 'F': 1008, 'G': 374}}])
130
A
A
130
1
1264


In [56]:
# total distance if round trips with one stop

def base_total_cost(adict):
    sum = 0
    for key in adict:
        sum += 2 * adict[key]
    return sum

base_tc = base_total_cost(distances_dc)

In [57]:
class Saving(object):

    def __init__(self, start, end, saving):
        self.nodes = (start, end)
        self.saving = saving
        self.used = False  # will be modified by algorithm

    def __lt__(self, other):
        return self.saving <= other.saving

    def __str__(self):
        return str(self.nodes) + ", " + str(self.saving)

savings = [Saving(key, nested_key,
                  distances_dc[key] + distances_dc[nested_key] - distances[key][nested_key])
           for key in distances for nested_key in distances[key]]

In [58]:
def sort(a_list, asc=True):
    """takes in a list of objects, returns it sorted"""
    for i in range(len(a_list)):
        for j in range(len(a_list)):
            if asc and a_list[j] > a_list[i]:
                a_list[j], a_list[i] = a_list[i], a_list[j]
            if not asc and a_list[j] < a_list[i]:
                a_list[j], a_list[i] = a_list[i], a_list[j]

    return a_list

savings = sort(savings, asc=False)

('B', 'E'), 1 ('C', 'F'), 4 ('A', 'D'), 12
